In [ ]:
!pip install --q git+https://github.com/m-bain/whisperx.git


  Preparing metadata (setup.py) ... done


In [ ]:
!pip install --upgrade pyannote.audio

In [ ]:
import whisperx
import gc

In [ ]:
device = "cpu"
batch_size = 16
compute_type = "float32"

In [ ]:

audio_file = "conversation.wav"


In [ ]:
import os
print(os.getcwd())  # Print the current working directory
print(os.path.exists(audio_file))  # Check if the file exists at the specified path

/content
True


In [ ]:
model = whisperx.load_model("large-v2", device , compute_type = compute_type)

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.80k [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

No language specified, language will be first be detected for each audio file (increases inference time).


100%|█████████████████████████████████████| 16.9M/16.9M [00:01<00:00, 10.9MiB/s]
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.5.1+cu121. Bad things might happen unless you revert torch to 1.x.


In [ ]:
audio = whisperx.load_audio(audio_file)
result = model.transcribe(audio, batch_size=batch_size)
print(result["segments"])

Detected language: en (0.99) in first 30s of audio...
[{'text': " You are brilliant, Hermione. Truly. Actually, I'm highly logical, which allows me to look past extraneous detail and perceive clearly that which others overlook. Yeah.", 'start': 0.031, 'end': 11.978}]


In [ ]:
result

{'segments': [{'text': " You are brilliant, Hermione. Truly. Actually, I'm highly logical, which allows me to look past extraneous detail and perceive clearly that which others overlook. Yeah.",
   'start': 0.031,
   'end': 11.978}],
 'language': 'en'}

In [ ]:
# Align whisper output
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

print(result["segments"])  #after alignment

Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960_asr_ls960.pth" to /root/.cache/torch/hub/checkpoints/wav2vec2_fairseq_base_ls960_asr_ls960.pth
100%|██████████| 360M/360M [00:01<00:00, 225MB/s]


[{'start': 0.351, 'end': 3.013, 'text': ' You are brilliant, Hermione.', 'words': [{'word': 'You', 'start': 0.351, 'end': 0.511, 'score': 0.918}, {'word': 'are', 'start': 0.571, 'end': 0.671, 'score': 0.919}, {'word': 'brilliant,', 'start': 1.352, 'end': 1.872, 'score': 0.833}, {'word': 'Hermione.', 'start': 2.572, 'end': 3.013, 'score': 0.452}]}, {'start': 3.473, 'end': 3.853, 'text': 'Truly.', 'words': [{'word': 'Truly.', 'start': 3.473, 'end': 3.853, 'score': 0.868}]}, {'start': 5.074, 'end': 11.178, 'text': "Actually, I'm highly logical, which allows me to look past extraneous detail and perceive clearly that which others overlook.", 'words': [{'word': 'Actually,', 'start': 5.074, 'end': 5.374, 'score': 0.81}, {'word': "I'm", 'start': 5.394, 'end': 5.514, 'score': 0.234}, {'word': 'highly', 'start': 6.135, 'end': 6.375, 'score': 0.889}, {'word': 'logical,', 'start': 6.415, 'end': 6.795, 'score': 0.83}, {'word': 'which', 'start': 6.835, 'end': 6.955, 'score': 0.916}, {'word': 'allow

In [ ]:
result

{'segments': [{'start': 0.351,
   'end': 3.013,
   'text': ' You are brilliant, Hermione.',
   'words': [{'word': 'You', 'start': 0.351, 'end': 0.511, 'score': 0.918},
    {'word': 'are', 'start': 0.571, 'end': 0.671, 'score': 0.919},
    {'word': 'brilliant,', 'start': 1.352, 'end': 1.872, 'score': 0.833},
    {'word': 'Hermione.', 'start': 2.572, 'end': 3.013, 'score': 0.452}]},
  {'start': 3.473,
   'end': 3.853,
   'text': 'Truly.',
   'words': [{'word': 'Truly.',
     'start': 3.473,
     'end': 3.853,
     'score': 0.868}]},
  {'start': 5.074,
   'end': 11.178,
   'text': "Actually, I'm highly logical, which allows me to look past extraneous detail and perceive clearly that which others overlook.",
   'words': [{'word': 'Actually,',
     'start': 5.074,
     'end': 5.374,
     'score': 0.81},
    {'word': "I'm", 'start': 5.394, 'end': 5.514, 'score': 0.234},
    {'word': 'highly', 'start': 6.135, 'end': 6.375, 'score': 0.889},
    {'word': 'logical,', 'start': 6.415, 'end': 6.795

In [ ]:
from google.colab import userdata
diarize_model = whisperx.DiarizationPipeline(use_auth_token=userdata.get('HF_TOKEN'), device=device)

In [ ]:
diarize_segments = diarize_model(audio, min_speakers= 2, max_speakers=2)

/usr/local/lib/python3.10/dist-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1823.)
  std = sequences.std(dim=-1, correction=1)


In [ ]:
diarize_segments

,segment,label,speaker,start,end
0,[ 00:00:00.300 --> 00:00:00.756],A,SPEAKER_01,0.300969,0.756594
1,[ 00:00:01.347 --> 00:00:02.005],B,SPEAKER_01,1.347219,2.005344
2,[ 00:00:02.545 --> 00:00:03.929],C,SPEAKER_01,2.545344,3.929094
3,[ 00:00:04.975 --> 00:00:05.616],D,SPEAKER_00,4.975344,5.616594
4,[ 00:00:05.970 --> 00:00:08.772],E,SPEAKER_00,5.970969,8.772219
5,[ 00:00:09.312 --> 00:00:11.151],F,SPEAKER_00,9.312219,11.151594
6,[ 00:00:11.151 --> 00:00:11.539],G,SPEAKER_01,11.151594,11.539719


In [ ]:
diarize_segments.speaker.unique()

array(['SPEAKER_01', 'SPEAKER_00'], dtype=object)

In [ ]:
!pip install pydub

In [ ]:
import pydub

# Load the audio file
audio = pydub.AudioSegment.from_wav(audio_file)

# Get unique speakers
speakers = diarize_segments.speaker.unique()

# Iterate over speakers
for speaker in speakers:
    # Filter segments for the current speaker
    speaker_segments = diarize_segments[diarize_segments['speaker'] == speaker]

    # Initialize an empty audio segment for the speaker
    merged_audio = pydub.AudioSegment.empty()

    # Iterate over speaker segments and merge
    for i in range(len(speaker_segments)):
        start = speaker_segments.iloc[i]['start']
        end = speaker_segments.iloc[i]['end']

        segment = audio[start * 1000 : end * 1000]
        merged_audio += segment

    # Save the merged audio for the speaker
    merged_audio.export(f"merged_{speaker}.wav", format="wav")

In [ ]:
!pip install pyannote.audio torch torchaudio

In [ ]:
import torchaudio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
from scipy.spatial.distance import cdist


def load_audio(file_path):
    waveform, sample_rate = torchaudio.load(file_path)
    return waveform, sample_rate

def extract_embedding(audio, sample_rate):
    embedding_model = PretrainedSpeakerEmbedding("speechbrain/spkrec-ecapa-voxceleb")
    waveform1, sample_rate1 = audio, sample_rate
    return embedding_model(waveform1[None])

def compare_embeddings(embedding1, embedding2):
    # Compute cosine similarity
    similarity = 1 - cdist(embedding1, embedding2, metric="cosine")
    return similarity[0][0]

def main(suspect_file, other_file):
    # Load audio files
    suspect_audio, suspect_sr = load_audio(suspect_file)
    other_audio, other_sr = load_audio(other_file)

    # Ensure sample rates are the same
    if suspect_sr != other_sr:
        raise ValueError("Sample rates of the audio files do not match")

    # Extract embeddings
    suspect_embedding = extract_embedding(suspect_audio, suspect_sr)
    other_embedding = extract_embedding(other_audio, other_sr)

    # Compare embeddings
    similarity = compare_embeddings(suspect_embedding, other_embedding)

    # Determine match threshold
    threshold = 0.8  # This is an example threshold; adjust based on your requirements


    if similarity >= threshold:
        print(f"Matched as the percentage of voice matched is: {similarity * 100:.2f}%")
    else:
        print(f"Not Matched as the percentage of voice matched is: {similarity * 100:.2f}%")

# Example usage
suspect_file = "speaker_2b.wav"
other_file = "merged_SPEAKER_00.wav"
main(suspect_file, other_file)


hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/autocast.py:68: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)


embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device

Not Matched as the percentage of voice matched is: 51.69%
